In [1]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00


In [2]:
!python -m pip install tensorflow

In [3]:
!pip install --upgrade nvidia-pyindex
!nvidia-smi

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=9a4200652b09f8f7fe06a77c883930d1978f94ebd4d769fe8316370f4bb5c4f4
  Stored in directory: /root/.cache/pip/wheels/2c/af/d0/7a12f82cab69f65d51107f48bcd6179e29b9a69a90546332b3
Successfully built nvidia-pyindex
/bin/bash: line 1: nvidia-smi: command not found


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# pulling in the data
import json

data_path = '/content/drive/MyDrive/chess_dataset_middle_100k.json'

with open(data_path, 'r') as file:
    data = json.load(file)

In [6]:
total_samples = len(data)
train_end = int(0.8 * total_samples)
val_end = int(0.9 * total_samples)

In [7]:
import numpy as np
import chess
from tensorflow.keras.utils import Sequence

In [8]:
#@title Binary Board
class BinaryBoard:

    # Constants for colors
    WHITE = 0
    BLACK = 1

    # BOARD MASKS

    ALL_DEFINED = 0xffffffffffffffff

    # BOUNDARIES

    NOT_A_FILE = 0xfefefefefefefefe
    NOT_H_FILE = 0x7f7f7f7f7f7f7f7f

    NOT_AB_FILE = 0xfcfcfcfcfcfcfcfc
    NOT_GH_FILE = 0x3f3f3f3f3f3f3f3f

    NOT_RANK_1 = ~0xff
    NOT_RANK_8 = ~0xff00000000000000

    def __init__(self, fen: str) -> None:
        self.piece_boards, self.board_data = self.fen_to_binary_boards(fen)

        self.combined_board = 0
        for bitboard in self.piece_boards.values():
            self.combined_board |= bitboard

    """
    Given a board and a square, returns the hex number at the square
    other returns the square on a board not self
    """

    def binary_to_piece(self, square, other=None) -> int:
        if other:
            return other >> (square & 0x1)
        return self.combined_board >> (square & 0x1)

    # FEN

    def fen_to_square_number(self, fen_square):
        if fen_square == "-":
            return -1

        files = "abcdefgh"
        ranks = "87654321"

        file = fen_square[0]
        rank = fen_square[1]

        file_index = files.index(file)
        rank_index = ranks.index(rank)

        square_number = rank_index * 8 + file_index

        return square_number

    def fen_to_binary_boards(self, fen: str) -> dict[str, int]:
        pieces = 'PNBRQKpnbrqk'
        boards = {piece: 0 for piece in pieces}

        split = fen.split(' ')
        positions = split[0]
        row = 0
        for rank in positions.split('/'):
            col = 0
            for char in rank:
                if char.isdigit():
                    col += int(char)
                else:
                    index = pieces.index(char)
                    board_index = 8 * (7 - row) + col
                    boards[char] |= 1 << board_index
                    col += 1
            row += 1

        color = 0 if split[1] == 'w' else 1
        castle_K = 1 if 'K' in split[2] else 0
        castle_Q = 1 if 'Q' in split[2] else 0
        castle_k = 1 if 'k' in split[2] else 0
        castle_q = 1 if 'q' in split[2] else 0
        en_passant = self.fen_to_square_number(
            split[3]) if split[3] != '-' else 0
        half_move = int(split[4])
        full_move = int(split[5]) if len(split) >= 5 else 0

        board_data = [color, castle_K, castle_Q, castle_k,
                      castle_q, en_passant, half_move, full_move]

        return boards, board_data

    # LINEAR MOVES

    @staticmethod
    def nortOne(b) -> int:
        return b << 8

    @staticmethod
    def soutOne(b) -> int:
        return b >> 8

    @staticmethod
    def eastOne(b) -> int:
        return (b << 1) & BinaryBoard.NOT_A_FILE

    # DIAG MOVES

    @staticmethod
    def noEaOne(b) -> int:
        return (b << 9) & BinaryBoard.NOT_A_FILE

    @staticmethod
    def soEaOne(b) -> int:
        return (b >> 7) & BinaryBoard.NOT_A_FILE

    @staticmethod
    def westOne(b) -> int:
        return (b >> 1) & BinaryBoard.NOT_H_FILE

    @staticmethod
    def soWeOne(b) -> int:
        return (b >> 9) & BinaryBoard.NOT_H_FILE

    @staticmethod
    def noWeOne(b) -> int:
        return (b << 7) & BinaryBoard.NOT_H_FILE

    @staticmethod
    def noNoEa(bitboard) -> int:
        return (bitboard << 17) & BinaryBoard.NOT_A_FILE

    # KNIGHT MOVES

    @staticmethod
    def noEaEa(bitboard) -> int:
        return (bitboard << 10) & BinaryBoard.NOT_AB_FILE

    @staticmethod
    def soEaEa(bitboard) -> int:
        return (bitboard >> 6) & BinaryBoard.NOT_AB_FILE

    @staticmethod
    def soSoEa(bitboard) -> int:
        return (bitboard >> 15) & BinaryBoard.NOT_A_FILE

    @staticmethod
    def noNoWe(bitboard) -> int:
        return (bitboard << 15) & BinaryBoard.NOT_H_FILE

    @staticmethod
    def noWeWe(bitboard) -> int:
        return (bitboard << 6) & BinaryBoard.NOT_GH_FILE

    @staticmethod
    def soWeWe(bitboard) -> int:
        return (bitboard >> 10) & BinaryBoard.NOT_GH_FILE

    @staticmethod
    def soSoWe(bitboard) -> int:
        return (bitboard >> 17) & BinaryBoard.NOT_H_FILE

    # SLIDE MOVES

    def _get_slide(self, bitboard, direction, boundary1, boundary2) -> int:
        next_boards = 0

        dir_bitboard: int = direction(bitboard)
        not_blocked = True

        while not_blocked and (dir_bitboard & boundary1) and (dir_bitboard & boundary2):
            next_boards |= dir_bitboard

            # update if hitting white or black piece to stop while loop
            not_blocked = dir_bitboard & self.combined_board == 0

            # moving up, down, left, right, or diag one position
            dir_bitboard = direction(dir_bitboard)

        added = False
        if dir_bitboard & ~boundary1 and not_blocked:
            next_boards |= dir_bitboard
            added = True

        if dir_bitboard & ~boundary2 and not_blocked and not added:
            next_boards |= dir_bitboard

        return next_boards

    def getNoSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.nortOne, BinaryBoard.NOT_RANK_8, BinaryBoard.ALL_DEFINED)

    def getSoSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.soutOne, BinaryBoard.NOT_RANK_1, BinaryBoard.ALL_DEFINED)

    def getEaSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.eastOne, BinaryBoard.NOT_H_FILE, BinaryBoard.ALL_DEFINED)

    def getWeSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.westOne, BinaryBoard.NOT_A_FILE, BinaryBoard.ALL_DEFINED)

    def getNoEaSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.noEaOne, BinaryBoard.NOT_RANK_8, BinaryBoard.NOT_H_FILE)

    def getNoWeSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.noWeOne, BinaryBoard.NOT_RANK_8, BinaryBoard.NOT_A_FILE)

    def getSoEaSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.soEaOne, BinaryBoard.NOT_RANK_1, BinaryBoard.NOT_H_FILE)

    def getSoWeSlide(self, bitboard) -> int:
        return self._get_slide(bitboard, BinaryBoard.soWeOne, BinaryBoard.NOT_RANK_1, BinaryBoard.NOT_A_FILE)

    # PIECE MOVES

    def _get_moves(self, bitboard: int, direction_fns) -> int:
        next_boards = 0

        for get_dir_boards in direction_fns:
            boards = get_dir_boards(bitboard)
            next_boards |= boards

        return next_boards

    def knight_moves(self, bitboard: int) -> int:
        directions = [self.noNoEa, self.noEaEa, self.noNoWe, self.noWeWe,
                      self.soSoEa, self.soEaEa, self.soSoWe, self.soWeWe]
        return self._get_moves(bitboard, directions)

    def rook_moves(self, bitboard: int) -> int:
        directions = [self.getNoSlide, self.getEaSlide,
                      self.getSoSlide, self.getWeSlide]
        return self._get_moves(bitboard, directions)

    def bishop_moves(self, bitboard: int) -> int:
        directions = [self.getNoEaSlide, self.getNoWeSlide,
                      self.getSoEaSlide, self.getSoWeSlide]
        return self._get_moves(bitboard, directions)

    def queen_moves(self, bitboard: int) -> int:
        directions = [self.getNoSlide, self.getEaSlide, self.getSoSlide, self.getWeSlide,  # rook moves
                      self.getNoEaSlide, self.getNoWeSlide, self.getSoEaSlide, self.getSoWeSlide]  # bishop moves
        return self._get_moves(bitboard, directions)

    def king_moves(self, bitboard: int) -> int:
        directions = [self.nortOne, self.soutOne, self.eastOne, self.westOne,
                      self.noEaOne, self.noWeOne, self.soEaOne, self.soWeOne]
        return self._get_moves(bitboard, directions)

    # ATTACKS

    def pawn_attacks_white(self, bitboard) -> int:
        attacks = 0

        noWeOne_bitboard = self.noWeOne(bitboard) & self.NOT_H_FILE
        noEaOne_bitboard = self.noEaOne(bitboard) & self.NOT_A_FILE

        attacks |= noWeOne_bitboard
        attacks |= noEaOne_bitboard

        return attacks

    def pawn_attacks_black(self, bitboard) -> int:
        attacks = 0

        soWeOne_bitboard = self.soWeOne(bitboard) & self.NOT_H_FILE
        soEaOne_bitboard = self.soEaOne(bitboard) & self.NOT_A_FILE

        attacks |= soWeOne_bitboard
        attacks |= soEaOne_bitboard

        return attacks

    # MAIN FUNCTIONS

    def generate_attack_board(self, color_to_look_for) -> int:
        attack_board = 0x0

        if color_to_look_for == self.WHITE:
            attack_board |= self.rook_moves(self.piece_boards['R'])
            attack_board |= self.knight_moves(self.piece_boards['N'])
            attack_board |= self.bishop_moves(self.piece_boards['B'])
            attack_board |= self.queen_moves(self.piece_boards['Q'])
            attack_board |= self.king_moves(self.piece_boards['K'])
            attack_board |= self.pawn_attacks_white(self.piece_boards['P'])
        else:
            attack_board |= self.rook_moves(self.piece_boards['r'])
            attack_board |= self.knight_moves(self.piece_boards['n'])
            attack_board |= self.bishop_moves(self.piece_boards['b'])
            attack_board |= self.queen_moves(self.piece_boards['q'])
            attack_board |= self.king_moves(self.piece_boards['k'])
            attack_board |= self.pawn_attacks_black(self.piece_boards['p'])

        return attack_board & self.ALL_DEFINED

    """
    Output: 22x8x8 (last 2 are attack boards for white and black, resp.)
    """

    def generate_board_matrix(self) -> list[list[list[int]]]:

        boards = [self.piece_boards['R'],
                  self.piece_boards['N'],
                  self.piece_boards['B'],
                  self.piece_boards['Q'],
                  self.piece_boards['K'],
                  self.piece_boards['P'],
                  self.piece_boards['r'],
                  self.piece_boards['n'],
                  self.piece_boards['b'],
                  self.piece_boards['q'],
                  self.piece_boards['k'],
                  self.piece_boards['p'],
                  self.generate_attack_board(0),  # white attack board
                  self.generate_attack_board(1)  # black attack board
                  ]

        boards += self.board_data

        matrix = []
        for board in boards:
            matrix.append(BinaryBoard.bitboard_to_matrix(board))

        return matrix

      # UTILS

    @staticmethod
    def bitboard_to_matrix(bitboard: int) -> list[list[int]]:
        matrix = []
        for row in range(8):
            current_row = []
            for col in range(8):
                index = (7 - row) * 8 + col
                if bitboard & (1 << index):
                    current_row.append(1)
                else:
                    current_row.append(0)
            matrix.append(current_row)
        return matrix

In [9]:
#@title Data Generator
class ChessDataGenerator(Sequence):
    def __init__(self, data, batch_size=32, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.data[i] for i in indices]
        X, y = self.__data_generation(batch)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch):
        X = np.empty((self.batch_size, 22, 8, 8))
        y = np.empty((self.batch_size), dtype=float)

        for i, item in enumerate(batch):
            X[i,] = self.fen_to_matrix(item[0])
            y[i] = item[1] / 1000.0

        return X, y

    def fen_to_matrix(self, fen):
        board = BinaryBoard(fen)
        return board.generate_board_matrix()

In [10]:
train_data = data[:train_end]
val_data = data[train_end:val_end]
test_data = data[val_end:]

train_generator = ChessDataGenerator(train_data, batch_size=32, shuffle=True)
val_generator = ChessDataGenerator(val_data, batch_size=32, shuffle=False)
test_generator = ChessDataGenerator(test_data, batch_size=32, shuffle=False)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

In [12]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(22, 8, 8)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='linear')
])

In [13]:
# Step 5: Compile the Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [15]:
# Train the Model
history = model.fit(train_generator, epochs=12, validation_data=val_generator)

Epoch 1/12
35476/35476 [==============================] - 1379s 39ms/step - loss: 0.0358 - mae: 0.1346 - val_loss: 0.0330 - val_mae: 0.1284
Epoch 2/12
35476/35476 [==============================] - 1358s 38ms/step - loss: 0.0299 - mae: 0.1210 - val_loss: 0.0289 - val_mae: 0.1175
Epoch 3/12
35476/35476 [==============================] - 1351s 38ms/step - loss: 0.0282 - mae: 0.1164 - val_loss: 0.0284 - val_mae: 0.1162
Epoch 4/12
35476/35476 [==============================] - 1371s 39ms/step - loss: 0.0273 - mae: 0.1141 - val_loss: 0.0285 - val_mae: 0.1153
Epoch 5/12
35476/35476 [==============================] - 1397s 39ms/step - loss: 0.0267 - mae: 0.1126 - val_loss: 0.0264 - val_mae: 0.1105
Epoch 6/12
35476/35476 [==============================] - 1360s 38ms/step - loss: 0.0263 - mae: 0.1115 - val_loss: 0.0263 - val_mae: 0.1105
Epoch 7/12
35476/35476 [==============================] - 1372s 39ms/step - loss: 0.0260 - mae: 0.1105 - val_loss: 0.0264 - val_mae: 0.1104
Epoch 8/12
35476/354

In [16]:
# Evaluate the Model
test_loss, test_mae = model.evaluate(test_generator)
print(f"Test MAE: {test_mae}")

1923/4434 [============>.................] - ETA: 1:19 - loss: 0.0276 - mae: 0.1122

KeyboardInterrupt: 

In [17]:
model.save('/content/drive/MyDrive/saved_models/chess_model_mid.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
